In [ ]:
import numpy as np
import pandas as pd
import igraph as ig
import vmlab

In [ ]:
from vmlab.models import fruit_model
fruit_model = fruit_model.drop_processes(['fruit_quality','fruit_composition'])
fruit_model.visualize()

In [ ]:
tree = pd.DataFrame({
    'parent_id': [np.nan, 0, 0],
    'id': [0, 1, 2],
    'cycle': [3, 3, 3],
    'is_apical': [1, 0, 1],
    'appearance_month': [2, 2, 2],
    'ancestor_nature': [0, 0, 0],
    'ancestor_is_apical': [1, 1, 1],
    'nature': [0, 0, 0],
    'burst_date': ['NAT', 'NAT', 'NAT'],
    'flowering_date': ['NAT', '2002-10-01', '2002-09-01'],
    'has_apical_child': [1, 0, 0],
    'nb_lateral_children': [1, 0, 0],
    'nb_inflo': [0, 1, 2],
    'nb_fruit': [0, 1, 1]
    
})
in_g = ig.Graph.DataFrame(tree.dropna())
ig.plot(in_g, bbox=(0,0,150,150), layout=in_g.layout_reingold_tilford())

In [ ]:
setup = vmlab.create_setup(
    model=fruit_model,
    tree=tree,
    start_date='2002-06-01',
    end_date='2003-06-01',
    setup_toml='../share/setup/fruit_model.toml',
    current_cycle=3,
    input_vars={
        'topology': {
            'seed': 11,
        },
        'geometry__interpretation_freq': 3
    },
    output_vars={
        'topology': {
            'adjacency': 'day'
        },
        'environment': {
            'TM': 'day',
            'TM_day': 'day',
            'GR': 'day'
        },
        'photosynthesis': {
            'photo': 'day',
            'Pmax': 'day',
            'D_fruit_avg': 'day'
        },
        'phenology': {
            'inflo_stage': 'day',
            'nb_fruit': 'day',
            'DAFB': 'day',
            'fruit_growth_tts': 'day'
        },
        'carbon_demand': {
            'D_fruit': 'day',
            'MR_stem': 'day',
            'MR_leaf': 'day',
            'MR_fruit': 'day',
            
        },
        'carbon_reserve': {
            'DM_structural_stem': 'day',
            'DM_structural_leaf': 'day',
            'reserve_stem': 'day',
            'reserve_leaf': 'day',
            'reserve_mob': 'day',
            'reserve_nmob_stem': 'day',
            'reserve_nmob_leaf': 'day',
            'reserve_leaf_max': 'day'
        },
        'carbon_allocation': {
            'carbon_supply': 'day',
            'DM_fruit': 'day',
            'reserve_stem_delta': 'day',
            'reserve_leaf_delta': 'day',
            'reserve_nmob_stem_delta': 'day',
            'reserve_nmob_leaf_delta': 'day',
            'remains_1': 'day',
            'required_DM_fruit': 'day'
        },
        'harvest': {
            'ripeness_index': 'day',
            'nb_fruit_harvested': None
        },
        'growth': {
            'nb_leaf': None,
        }
     }
)

In [ ]:
ds_out = vmlab.run(setup, fruit_model, geometry=True)

In [ ]:
g = ig.Graph.Adjacency([row.tolist() for row in ds_out.topology__adjacency[-1].data.astype(np.int64)])
layout = g.layout_reingold_tilford(root=0.)
layout.rotate(-180)
ig.plot(g, layout=layout, bbox=(600, 300), **{
    'vertex_size': 1,
    'vertex_label_size': 10,
    'edge_arrow_width': 0.1,
    'vertex_label': [
        f'GU{idx}\nF:{int(ds_out.harvest__nb_fruit_harvested.data[idx])}\nL:{int(ds_out.growth__nb_leaf.data[idx])}' for idx in g.vs.indices
    ]
})

In [ ]:
ds_out.phenology__fruit_growth_tts.plot(col='GU', col_wrap=6)

In [ ]:
ds_out.carbon_allocation__DM_fruit.plot(col='GU', col_wrap=6)

In [ ]:
ds_out.carbon_demand__D_fruit.plot(col='GU', col_wrap=6)

In [ ]:
ds_out.photosynthesis__D_fruit_avg.plot(col='GU', col_wrap=6)

In [ ]:
ds_out.photosynthesis__Pmax.plot(col='GU', col_wrap=6)

In [ ]:
ds_out.photosynthesis__photo.plot(col='GU', col_wrap=6)